In [ ]:
import re
from pathlib import Path
from asr_eval.align.parsing import parse_multivariant_string
from asr_eval.align.data import MultiVariant
import numpy as np

In [ ]:
car_words = '{джили|geeley}, {тойота|тёха|toyota}, {хендай|хёндай|hyundai}, {киа|kia}, {форд|фурда|ford}, {хонда|honda}, {ниссан|нисан|nissan}, {мерседес|мерс|мерин|mercedes}, {бмв|бэха|бумер|bmw}, {ауди|audi}, {фольксваген|фольц|жук|volkswagen}, {шкода|skoda}, {рено|reno}, {пежо|peugeot}, {ситроен|ситро|citroen}, {шевроле|шевик|chevrolet}, {мазда|mazda}, {субару|субарик|subaru}, {мицубиси|мицуба|мицубиши|mitsubishi}, {лексус|лекс|lexus}, {инфинити|infiniti}, {вольво|volvo}, {опель|opel}, {порше|порш|porsche}, {черри|chery}, {брэнд|бриллианс|brilliance}, {чана|chana}, {фотон|foton}, {зотье|zotye}, {лада|копейка|жигули|ваз|автоваз|lada}, {уаз|буханка|козёл|uaz}, {газ|волга|газик|gaz}, {москвич|moskvich}, {таграз|tagaz}, {лифан|lifan}, {хавал|haval}, {джей эм си|jmc}, {байк|baic}, {джак|jac}, {линкольн|линк|lincoln}, {додж|доджик|dodge}, {джип|jeep}, {бьюик|буик|buick}, {кадиллак|кадик|cadillac}, {лэнд ровер|лендрик|ровер|land rover}, {ягуар|яга|jaguar}, {мини|миник|миникупер|mini}, {смарт|смартик|smart}, {сааб|saab}'
place_words = '''
Абс Авто (Land Rover), Сочи, ул. Ленина, стр. 286,
Абс Авто (Jaguar), Сочи, ул. Ленина, стр. 286,
Автоимпорт (Hyundai), Тула, ул. Октябрьская, д. 320А,
Фольксваген Авилон Белая Дача (Volkswagen), МО, Котельники, Коммерческий пр-д, 10,
Башавтоком (Suzuki), Стерлитамак, ул. Космонавтов, д. 1,
СТК ЦЕНТР (Nissan), Санкт-Петербург, пр-т Косыгина, д. 2, к. 1А,
Автомир Архангельск (Nissan), Архангельск, Московский пр-т, д. 35,
Автомир Сургут (Renault), Сургут, ул. Маяковского, д. 42,
Автопорт (BMW), Москва, Пятницкое ш., 6-й км,
Порше Центр Ясенево (Porsche), Москва, ул. Ясногорская, вл. 3,
БалтАвтоТрейд-М (BMW), Москва, ул. Лобненская, д. 17,
Бакра (BMW), Москва, ул. Дорожная, д. 3,
Автоцентр Чешский дом (Skoda), Москва, ул. Марксистская, д. 34, к. 7,
АВТО-БЕЛОГОРЬЕ (Renault), Белгород, ул. Корочанская, д. 130,
Sollers Динамика, Архангельск, пр-т Московский, д. 39,
Sollers Автоград, Астрахань, ул. Боевая, д. 135,
Sollers АлтайАвтоЦентр, Барнаул, Павловский тракт, д. 249,
Sollers Интеркар, Белгород, р-т Б. Хмельницкого, 205А,
HONDA БашАвтоком, Уфа, ул. Салавата Юлаева, 89,
HONDA Беляево, Москва, ул. Академика Волгина, 6А, стр. 3,
HONDA Боровское, Москва, МКАД 48 км, ул. Домостроительная, 4,
HONDA Бутово, Москва, Чечерский проезд, 1,
Honda Звезда Невы, Санкт-Петербург, Витебский пр., д. 17, корпус 1,
HONDA Лосиный остров, Москва, Ярославское ш., вл2Вс1,
HONDA на Федюнинского, Тюмень, Федюнинского, 10А,
Honda на Шоссе Энтузиастов, Москва, ул. Плеханова, 5,
HONDA Новая Рига, Москва, М-9 Балтия, вл24,
Honda Самара, Самара, Московское шоссе, 264,
HONDA Строгино, Москва, ул. Маршала Прошлякова, 13,
Honda Тверь, Тверь, Московское ш., д. 1, корп. 1.
Рольф Хёндай (Hyundai), Москва, Ленинградский пр-т, 57
Авто Премиум (Lexus), Санкт-Петербург, пр-т Энергетиков, 3
Горький Авто (Volkswagen), Нижний Новгород, ул. Коминтерна, 117
Авилон БМВ (BMW), Екатеринбург, ул. Репина, 94
Мерседес-Бенц Рязань (Mercedes), Рязань, ул. Солнечная, 37
Тойота Центр Казань (Toyota), Казань, ул. Аделя Кутуя, 120
Фаворит Моторс (Kia), Краснодар, ул. Ростовское шоссе, 27
АвтоСпецЦентр Форд (Ford), Воронеж, Московский пр-т, 131Б
Лада Центр Урал (Lada), Челябинск, ул. Машиностроителей, 29
Хавъл Моторс (Haval), Новосибирск, ул. Ватутина, 45
Ниссан Арена (Nissan), Самара, ул. Ново-Садовая, 160
Рено Арктика (Renault), Мурманск, ул. Книповича, 33
Порше Центр Урал (Porsche), Уфа, ул. Менделеева, 137
Ауди Центр Сибирь (Audi), Омск, ул. 10 лет Октября, 195
Субару ДВ (Subaru), Хабаровск, ул. Карла Маркса, 64
Мазда Восток (Mazda), Владивосток, ул. Русская, 55
Инфинити Москва (Infiniti), Москва, ул. Нижняя Масловка, 12
Чери Авто (Chery), Ростов-на-Дону, ул. Металлургическая, 102
Джили Таганрог (Geely), Таганрог, ул. Петровская, 89
БМВ Тула (BMW), Тула, ул. Октябрьская, 81
Фольксваген Калининград (Volkswagen), Калининград, ул. Генерала Челнокова, 12
Опель Иркутск (Opel), Иркутск, ул. Ржанова, 164
Вольво Архангельск (Volvo), Архангельск, ул. Терехина, 8
Мицубиси Ярославль (Mitsubishi), Ярославль, ул. Полушкина Роща, 15
Ситроен Самара (Citroën), Самара, ул. Стара-Загора, 18
Пежо Урал (Peugeot), Пермь, ул. Промышленная, 112
Шкода Сибирь (Skoda), Томск, ул. Иркутский тракт, 47
Джип Плаза (Jeep), Сочи, ул. Донская, 123
Кадиллак Москва (Cadillac), Москва, ул. Лобненская, 4
Лэнд Ровер Екатеринбург (Land Rover), Екатеринбург, ул. Репина, 94
'''

In [ ]:
word_to_multivariant: dict[str, str] = {}

for block in car_words.split(', '):
    assert block.startswith('{')
    assert block.endswith('}')
    for option in block[1:-1].split('|'):
        assert option not in word_to_multivariant
        word_to_multivariant[option] = block

word_to_multivariant

In [ ]:
words = set([
    word.lower() for word in re.findall(r'\w+', car_words + ' ' + place_words) if not re.search(r'[0-9]', word) and len(word) > 2
])

', '.join(sorted(words))

In [ ]:
text = Path('datasets/small-domain-dicts/cars/texts.txt').read_text()
blocks = re.split(r'\n\n', text, flags=re.DOTALL)
blocks = [b.split('\n', 2)[-1] for b in blocks]
blocks = [
    b.strip()
    .replace('\n', ' ')
    .replace('### ', '')
    .replace('**', '')
    .replace('— ', '')
    .replace('»', '')
    .replace('«', '')
    .replace('«', '')
    .replace('"', '')
    .replace('~', '')
    for b in blocks
]
blocks = [re.sub(r'\s+\n', '\n', b) for b in blocks]
blocks = [re.sub(r'\s+', ' ', b) for b in blocks]

for i, block in enumerate(blocks):
    items = list(word_to_multivariant.items())
    for i2, (k, v) in enumerate(items):
        block = block.replace(k, f'<<{i2}>>')
    for i2 in range(len(items)):
        block = block.replace(f'<<{i2}>>', items[i2][1])
    blocks[i] = block

for block in blocks:
    print(block)

In [ ]:
# сохранить обратно тексты
# вручную убрать места с цифрами

In [ ]:
def is_cyrillic(text: str):
    for char in text:
        if not (0x0400 <= ord(char) <= 0x04FF) and char != ' ':
            return False
    return True

lines = Path('datasets/small-domain-dicts/cars/texts_multivariant.txt').read_text().splitlines()
for line in lines:
    tokens = parse_multivariant_string(line)
    for t in tokens[::-1]:
        if isinstance(t, MultiVariant):
            start, end = t.pos
            orig_options = [' '.join(str(t.value) for t in option) for option in t.options]
            options = [x for x in orig_options if is_cyrillic(x)]
            option = np.random.choice(options)
            line = line[:start] + option + line[end:]
    line = line.replace(' .', '.').replace(' ,', ',').replace(' !', '!').replace(' ?', '?')
    print(line)